# Step 4: Isolation Forest Variants

**Learning Objectives:**
1. Understand how Isolation Forest works (unsupervised anomaly detection)
2. Implement Standard Isolation Forest
3. Compare with Random Forest results
4. Understand contamination parameter
5. Analyze sensitivity vs specificity tradeoffs

---

## What is Isolation Forest?

**Key Concept:** Anomalies are "easier to isolate" than normal points.

**How it works:**
- Build random decision trees
- Anomalies get isolated with fewer splits
- Gives each point an "anomaly score"
- Higher score = more likely to be anomaly

**Key Parameter:**
- `contamination`: Expected proportion of anomalies (we use 0.101 = 10.1%)

In [2]:
# load the data and the libraries

import pandas as pd
import numpy as np 
import pickle
from sklearn.ensemble import IsolationForest 
from sklearn.metrics import confusion_matrix, accuracy_score 
import time 

# Seed
random_seed = 123
np.random.seed(random_seed)

# Load preprocessed data
with open('preprocessed_data.pkl', 'rb') as f:
    data = pickle.load(f)

# Extract the needed data 
X_train_scaled = data['X_train_scaled']  # Isolation Forest needs scaled data!
X_test_scaled = data['X_test_scaled']
y_train = data['y_train']
y_test = data['y_test']
contamination_rate = data['contamination_rate']

In [3]:
# Train Standard Isolation Forest

iso_forest = IsolationForest(
    n_estimators = 100,            # Number of trees
    max_samples = 1000,            # Sample size per tree
    contamination = contamination_rate, # Expected proportion of anomalies
    random_state = random_seed,
    n_jobs = -1,
    verbose = 1
)

# Train the model 
start_time = time.time()

iso_forest.fit(X_train_scaled)

training_time = time.time()-start_time
print(f"Training model completed in {training_time:.2f} seconds")

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.9s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished


Training model completed in 0.39 seconds


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [4]:
# Making prediction 

# Predict on test set
# Returns -1 for anomalies, 1 for normal - Sklearn convention 
y_pred_if_raw = iso_forest.predict(X_test_scaled)

# Convert to our format: 0 = Normal, 1 = Anomaly 
# sklearn: 1=normal, -1=anomaly
# We need: 0=normal, 1=anomaly
y_pred_if = np.where(y_pred_if_raw == -1, 1, 0)

# Counting prediction 
unique, counts = np.unique(y_pred_if, return_counts = True)
pred_dict = dict(zip(unique, counts))

print(f"\n Predictions:")
print(f"  Normal (0):   {pred_dict.get(0, 0):,}")
print(f"  Abnormal (1): {pred_dict.get(1, 0):,}")

# Expected abnormal based on contamination
expected_abnormal = int(len(y_test) * contamination_rate)
print(f"\nExpected abnormal (based on contamination): {expected_abnormal:,}")
print(f"Actual predicted abnormal: {pred_dict.get(1, 0):,}")



 Predictions:
  Normal (0):   39,691
  Abnormal (1): 4,703

Expected abnormal (based on contamination): 4,733
Actual predicted abnormal: 4,703


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [5]:
# Calculate performance matricies 
from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate metrics 
accuracy = accuracy_score(y_test, y_pred_if)

# Confusion matrix 
conf_matrix = confusion_matrix(y_test, y_pred_if)
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate sensitivity and specificity 
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Actual Normal    {tn:6d}  {fp:6d}")
print(f"       Abnormal  {fn:6d}  {tp:6d}")

print(f"\n{'='*70}")
print("METRICS:")
print(f"{'='*70}")
print(f"Accuracy:    {accuracy*100:.2f}%")
print(f"Sensitivity: {sensitivity*100:.2f}%  (catches {sensitivity*100:.1f}% of abnormal)")
print(f"Specificity: {specificity*100:.2f}%  (catches {specificity*100:.1f}% of normal)")

print(f"\n{'='*70}")
print("COMPARISON:")
print(f"{'='*70}")
print(f"Random Forest:      Acc={98.14:.2f}%, Sens={84.62:.2f}%, Spec={99.75:.2f}%")
print(f"Isolation Forest:   Acc={accuracy*100:.2f}%, Sens={sensitivity*100:.2f}%, Spec={specificity*100:.2f}%")

Actual Normal     36689    2972
       Abnormal    3002    1731

METRICS:
Accuracy:    86.54%
Sensitivity: 36.57%  (catches 36.6% of abnormal)
Specificity: 92.51%  (catches 92.5% of normal)

COMPARISON:
Random Forest:      Acc=98.14%, Sens=84.62%, Spec=99.75%
Isolation Forest:   Acc=86.54%, Sens=36.57%, Spec=92.51%



## Single-Variable Isolation Forest

**Key Idea:** Instead of one model using all 150 features, train 150 separate models - one per feature!

**How it works:**
1. Train 150 individual Isolation Forests (one for each R-R interval feature)
2. Each model gives an anomaly score
3. Average all 150 scores
4. Higher average score = more likely anomaly


**Expected from R:** Sensitivity 92.52%, Specificity 31.45%

In [6]:
# Train 150 models - One per feature 

n_features = X_train_scaled.shape[1] 
single_models = []
feature_scores = np.zeros((X_test_scaled.shape[0], n_features))

print(f"Training {n_features} individual isolation forest models.")

start_time = time.time()

for i in range(n_features):
    if(i + 1) % 30 == 0:      # Progress update every 30 models
        print(f" Processed {i+1} / {n_features} features.")
    
    # Training IF on single feature
    single_if = IsolationForest(
        n_estimators = 20,           # 20 Trees per model
        max_samples = 500,
        contamination = contamination_rate,
        random_state = random_seed
    )

    # Fit on a single feature
    X_train_single = X_train_scaled[:, i].reshape(-1,1)
    single_if.fit(X_train_single)

    # Predict on test set for this feature
    X_test_single = X_test_scaled[:, i].reshape(-1,1)
    predictions = single_if.predict(X_test_single)

    # Convert to 0/1 and store
    feature_scores[:, i] = np.where(predictions == -1, 1, 0)
    single_models.append(single_if)

training_time = time.time() - start_time

print(f"Training completed in {training_time:.2f} seconds.")
print(f"Total trees: {n_features * 20} = {n_features * 20}")

Training 150 individual isolation forest models.
 Processed 30 / 150 features.
 Processed 60 / 150 features.
 Processed 90 / 150 features.
 Processed 120 / 150 features.
 Processed 150 / 150 features.
Training completed in 15.59 seconds.
Total trees: 3000 = 3000


In [9]:
# Average the pridictions across all 150 features 
# Eech raw = one test sample
# Each column = one feature's vote (0 or 1)
# Average = What % of features say 'abnormal'"ECG R TO PY.ipynb"

average_scores = feature_scores.mean(axis=1)

print(f"Average scores for test samples: {len(average_scores):,}")

# Determine threshold based on contamination
# If contamination = 0.101, we want top 10.1% to be labled abnormal 

threshold = np.quantile(average_scores, 1 - contamination_rate)

# Final prediction
y_pred_single_var = np.where(average_scores >= threshold, 1, 0)

# Count predictions
pred_normal = sum(y_pred_single_var == 0)
pred_abnormal = sum (y_pred_single_var == 1)

print(f"\nFinal Predictions:")
print(f"  Normal (0):   {pred_normal:,}")
print(f"  Abnormal (1): {pred_abnormal:,}")



Average scores for test samples: 44,394

Final Predictions:
  Normal (0):   39,645
  Abnormal (1): 4,749


In [11]:
# Calculate metrics 
accuracy_sv = accuracy_score(y_test, y_pred_single_var)

# confusion matrix 
cm_sv = confusion_matrix(y_test, y_pred_single_var)
tn_sv, fn_sv, fp_sv, tp_sv = cm_sv.ravel()

# Calculate sensitivity and specificity
sensitivity_sv = tp_sv / (tp_sv + fn_sv)
specificity_sv = tn_sv / (tn_sv + fp_sv)

print(f"Actual Normal    {tn_sv:6d}  {fp_sv:6d}")
print(f"       Abnormal  {fn_sv:6d}  {tp_sv:6d}")

print(f"Accuracy:    {accuracy_sv*100:.2f}%")
print(f"Sensitivity: {sensitivity_sv*100:.2f}%  (catches {sensitivity_sv*100:.1f}% of abnormal)")
print(f"Specificity: {specificity_sv*100:.2f}%  (catches {specificity_sv*100:.1f}% of normal)")

print(f"R (from paper):")
print(f"  Single-Variable IF:  Acc=85.70%, Sens=92.52%, Spec=31.45%")
print(f"\nPython (current):")
print(f"  Random Forest:       Acc=98.14%, Sens=84.62%, Spec=99.75%")
print(f"  Standard IF:         Acc=86.54%, Sens=36.57%, Spec=92.51%")
print(f"  Single-Variable IF:  Acc={accuracy_sv*100:.2f}%, Sens={sensitivity_sv*100:.2f}%, Spec={specificity_sv*100:.2f}%")

Actual Normal     36241    3404
       Abnormal    3420    1329
Accuracy:    84.63%
Sensitivity: 27.98%  (catches 28.0% of abnormal)
Specificity: 91.41%  (catches 91.4% of normal)
R (from paper):
  Single-Variable IF:  Acc=85.70%, Sens=92.52%, Spec=31.45%

Python (current):
  Random Forest:       Acc=98.14%, Sens=84.62%, Spec=99.75%
  Standard IF:         Acc=86.54%, Sens=36.57%, Spec=92.51%
  Single-Variable IF:  Acc=84.63%, Sens=27.98%, Spec=91.41%


---

## SCiForest (Sparse-Cut Isolation Forest)

**Key Idea:** Use random subsets of features for each model

**How it works:**
1. Train multiple models (we'll use 50 models)
2. Each model uses only 75 random features (out of 150)
3. Each model has 10 trees
4. Average all anomaly scores

**Why this might work:**
- Reduces feature dominance
- Forces diversity across models
- Feature sparsity can improve generalization

**Expected from R:** Accuracy 84.68%, Sensitivity 90.17%, Specificity 30.98%

In [13]:
print("="*70)
print("SCIFOREST (SPARSE-CUT ISOLATION FOREST)")
print("="*70)

n_models = 50
n_features_per_model = 75  # Use half of 150 features

print(f"\nTraining {n_models} models...")
print(f"Each model uses {n_features_per_model} random features")
print(f"Each model has 10 trees")

sci_scores = np.zeros(X_test_scaled.shape[0])

start_time = time.time()

for i in range(n_models):
    if (i + 1) % 10 == 0:
        print(f"  Training model {i + 1}/{n_models}...")
    
    # Randomly select 75 features
    selected_features = np.random.choice(150, n_features_per_model, replace=False)
    
    # Train model on selected features
    sci_model = IsolationForest(
        n_estimators=10,  # 10 trees per model
        max_samples=1000,
        contamination=contamination_rate,
        random_state= random_seed + i  # Different seed for each model
    )
    
    # Fit on training data with selected features
    sci_model.fit(X_train_scaled[:, selected_features])
    
    # Predict on test data with same features
    predictions = sci_model.predict(X_test_scaled[:, selected_features])
    
    # Convert and accumulate scores
    sci_scores += np.where(predictions == -1, 1, 0)

training_time = time.time() - start_time

# Average scores across all models
sci_scores = sci_scores / n_models

# Apply threshold
threshold = np.quantile(sci_scores, 1 - contamination_rate)
y_pred_sci = np.where(sci_scores >= threshold, 1, 0)

print(f"\n✓ Training complete in {training_time:.2f} seconds")
print(f"✓ Total: {n_models} models × 10 trees = {n_models * 10} trees")

# Evaluate
accuracy_sci = accuracy_score(y_test, y_pred_sci)
cm_sci = confusion_matrix(y_test, y_pred_sci)
tn_sci, fp_sci, fn_sci, tp_sci = cm_sci.ravel()
sensitivity_sci = tp_sci / (tp_sci + fn_sci)
specificity_sci = tn_sci / (tn_sci + fp_sci)

print(f"\n{'='*70}")
print("SCIFOREST PERFORMANCE:")
print(f"{'='*70}")
print(f"Accuracy:    {accuracy_sci*100:.2f}%")
print(f"Sensitivity: {sensitivity_sci*100:.2f}%")
print(f"Specificity: {specificity_sci*100:.2f}%")
print(f"\nR result: Acc=84.68%, Sens=90.17%, Spec=30.98%")

SCIFOREST (SPARSE-CUT ISOLATION FOREST)

Training 50 models...
Each model uses 75 random features
Each model has 10 trees
  Training model 10/50...
  Training model 20/50...
  Training model 30/50...
  Training model 40/50...
  Training model 50/50...

✓ Training complete in 4.38 seconds
✓ Total: 50 models × 10 trees = 500 trees

SCIFOREST PERFORMANCE:
Accuracy:    86.25%
Sensitivity: 36.04%
Specificity: 92.24%

R result: Acc=84.68%, Sens=90.17%, Spec=30.98%


---

## Density-Aware Isolation Forest

**Key Idea:** Weight samples by local density - favor less dense regions

**How it works:**
1. Calculate density for each training sample (using k-nearest neighbors)
2. Sample training data with bias toward low-density regions
3. Train 100 models, each on differently sampled data
4. Average anomaly scores

**Expected from R:** Lower performance (specificity issues)

In [15]:
print("="*70)
print("DENSITY-AWARE ISOLATION FOREST")
print("="*70)

from sklearn.neighbors import NearestNeighbors

# Calculate density weights
print("\nCalculating local density for training samples...")
k = 10
nn = NearestNeighbors(n_neighbors=k)
nn.fit(X_train_scaled)
distances, _ = nn.kneighbors(X_train_scaled)
knn_distances = distances[:, -1]  # Distance to k-th neighbor

# Convert to density weights (inverse relation)
density_weights = 1 / (knn_distances + 1e-6)
density_weights = density_weights / density_weights.sum()

# Invert for sampling (favor low density)
sampling_weights = 1 / (density_weights + 1e-6)
sampling_weights = sampling_weights / sampling_weights.sum()

print(f"✓ Density weights calculated for {len(density_weights):,} samples")

# Train ensemble
n_trees = 100
density_scores = np.zeros(X_test_scaled.shape[0])

print(f"\nTraining {n_trees} models with density-based sampling...")
start_time = time.time()

for i in range(n_trees):
    if (i + 1) % 20 == 0:
        print(f"  Training tree {i + 1}/{n_trees}...")
    
    # Sample with density bias
    sample_indices = np.random.choice(
        len(X_train_scaled), 
        size=1000, 
        replace=True,
        p=sampling_weights
    )
    sample_data = X_train_scaled[sample_indices]
    
    # Train model
    density_model = IsolationForest(
        n_estimators=1,
        max_samples=1000,
        contamination=contamination_rate,
        random_state= random_seed + i
    )
    density_model.fit(sample_data)
    
    # Predict
    predictions = density_model.predict(X_test_scaled)
    density_scores += np.where(predictions == -1, 1, 0)

training_time = time.time() - start_time
density_scores = density_scores / n_trees

# Apply threshold
threshold = np.quantile(density_scores, 1 - contamination_rate)
y_pred_density = np.where(density_scores >= threshold, 1, 0)

print(f"\n✓ Training complete in {training_time:.2f} seconds")

# Evaluate
accuracy_dens = accuracy_score(y_test, y_pred_density)
cm_dens = confusion_matrix(y_test, y_pred_density)
tn_dens, fp_dens, fn_dens, tp_dens = cm_dens.ravel()
sensitivity_dens = tp_dens / (tp_dens + fn_dens)
specificity_dens = tn_dens / (tn_dens + fp_dens)

print(f"\n{'='*70}")
print("DENSITY-AWARE IF PERFORMANCE:")
print(f"{'='*70}")
print(f"Accuracy:    {accuracy_dens*100:.2f}%")
print(f"Sensitivity: {sensitivity_dens*100:.2f}%")
print(f"Specificity: {specificity_dens*100:.2f}%")

DENSITY-AWARE ISOLATION FOREST

Calculating local density for training samples...
✓ Density weights calculated for 44,394 samples

Training 100 models with density-based sampling...
  Training tree 20/100...
  Training tree 40/100...
  Training tree 60/100...
  Training tree 80/100...
  Training tree 100/100...

✓ Training complete in 1.30 seconds

DENSITY-AWARE IF PERFORMANCE:
Accuracy:    87.20%
Sensitivity: 44.64%
Specificity: 92.27%


---

## Boxed Isolation Forest

**Key Idea:** Create hierarchical feature boxes instead of using all features

**How it works:**
1. Train 100 models
2. Each model uses a random "box" of 10-30 consecutive features
3. Average all predictions

**Why:** Groups related features together (temporal locality in R-R intervals)

**Expected from R:** Poor specificity

In [16]:
print("="*70)
print("BOXED ISOLATION FOREST")
print("="*70)

n_boxes = 100
box_scores = np.zeros(X_test_scaled.shape[0])

print(f"\nTraining {n_boxes} models with random feature boxes...")
start_time = time.time()

for i in range(n_boxes):
    if (i + 1) % 20 == 0:
        print(f"  Training box {i + 1}/{n_boxes}...")
    
    # Create random box of features
    box_size = np.random.randint(10, 31)  # Random size 10-30
    max_start = 150 - box_size
    start_feature = np.random.randint(0, max_start + 1)
    box_features = list(range(start_feature, start_feature + box_size))
    
    # Train model on this feature box
    box_model = IsolationForest(
        n_estimators=1,
        max_samples=1000,
        contamination=contamination_rate,
        random_state=random_seed + i
    )
    box_model.fit(X_train_scaled[:, box_features])
    
    # Predict
    predictions = box_model.predict(X_test_scaled[:, box_features])
    box_scores += np.where(predictions == -1, 1, 0)

training_time = time.time() - start_time
box_scores = box_scores / n_boxes

# Apply threshold
threshold = np.quantile(box_scores, 1 - contamination_rate)
y_pred_boxed = np.where(box_scores >= threshold, 1, 0)

print(f"\n✓ Training complete in {training_time:.2f} seconds")

# Evaluate
accuracy_box = accuracy_score(y_test, y_pred_boxed)
cm_box = confusion_matrix(y_test, y_pred_boxed)
tn_box, fp_box, fn_box, tp_box = cm_box.ravel()
sensitivity_box = tp_box / (tp_box + fn_box)
specificity_box = tn_box / (tn_box + fp_box)

print(f"\n{'='*70}")
print("BOXED IF PERFORMANCE:")
print(f"{'='*70}")
print(f"Accuracy:    {accuracy_box*100:.2f}%")
print(f"Sensitivity: {sensitivity_box*100:.2f}%")
print(f"Specificity: {specificity_box*100:.2f}%")

BOXED ISOLATION FOREST

Training 100 models with random feature boxes...
  Training box 20/100...
  Training box 40/100...
  Training box 60/100...
  Training box 80/100...
  Training box 100/100...

✓ Training complete in 1.71 seconds

BOXED IF PERFORMANCE:
Accuracy:    85.50%
Sensitivity: 32.83%
Specificity: 91.79%


---

## Fair-Cut Forest

**Key Idea:** Weight features inversely by variance to ensure fair representation

**How it works:**
1. Calculate variance for each feature
2. Apply inverse variance weights (low variance features get higher weight)
3. Train 100 models on weighted features
4. Average predictions

**Expected from R:** Poor performance (this had the variance bug!)

In [17]:
print("="*70)
print("FAIR-CUT FOREST")
print("="*70)

# Calculate fair weights (inverse variance)
feature_vars = X_train_scaled.var(axis=0)
fair_weights = 1 / (feature_vars + 1e-6)
fair_weights = fair_weights / fair_weights.sum()

print(f"\nCalculated fair weights for {len(fair_weights)} features")
print(f"Weight range: {fair_weights.min():.6f} to {fair_weights.max():.6f}")

n_trees = 100
faircut_scores = np.zeros(X_test_scaled.shape[0])

print(f"\nTraining {n_trees} models with fair-weighted features...")
start_time = time.time()

for i in range(n_trees):
    if (i + 1) % 20 == 0:
        print(f"  Training tree {i + 1}/{n_trees}...")
    
    # Apply fair weights to features
    weighted_train = X_train_scaled * fair_weights
    weighted_test = X_test_scaled * fair_weights
    
    # Train model
    faircut_model = IsolationForest(
        n_estimators=1,
        max_samples=1000,
        contamination=contamination_rate,
        random_state=random_seed + i
    )
    faircut_model.fit(weighted_train)
    
    # Predict
    predictions = faircut_model.predict(weighted_test)
    faircut_scores += np.where(predictions == -1, 1, 0)

training_time = time.time() - start_time
faircut_scores = faircut_scores / n_trees

# Apply threshold
threshold = np.quantile(faircut_scores, 1 - contamination_rate)
y_pred_faircut = np.where(faircut_scores >= threshold, 1, 0)

print(f"\n✓ Training complete in {training_time:.2f} seconds")

# Evaluate
accuracy_fc = accuracy_score(y_test, y_pred_faircut)
cm_fc = confusion_matrix(y_test, y_pred_faircut)
tn_fc, fp_fc, fn_fc, tp_fc = cm_fc.ravel()
sensitivity_fc = tp_fc / (tp_fc + fn_fc)
specificity_fc = tn_fc / (tn_fc + fp_fc)

print(f"\n{'='*70}")
print("FAIR-CUT FOREST PERFORMANCE:")
print(f"{'='*70}")
print(f"Accuracy:    {accuracy_fc*100:.2f}%")
print(f"Sensitivity: {sensitivity_fc*100:.2f}%")
print(f"Specificity: {specificity_fc*100:.2f}%")

FAIR-CUT FOREST

Calculated fair weights for 150 features
Weight range: 0.006667 to 0.006667

Training 100 models with fair-weighted features...
  Training tree 20/100...
  Training tree 40/100...
  Training tree 60/100...
  Training tree 80/100...
  Training tree 100/100...

✓ Training complete in 4.76 seconds

FAIR-CUT FOREST PERFORMANCE:
Accuracy:    86.75%
Sensitivity: 38.01%
Specificity: 92.57%


In [18]:
print("="*70)
print("FINAL RESULTS - ALL MODELS")
print("="*70)

# Create results dataframe
results = {
    'Algorithm': [
        'Random Forest',
        'Standard IF',
        'Single-Variable IF',
        'SCiForest',
        'Density-Aware IF',
        'Boxed IF',
        'Fair-Cut Forest'
    ],
    'Accuracy (%)': [
        98.14,
        86.54,
        84.63,
        86.25,
        87.20,
        85.50,
        86.75
    ],
    'Sensitivity (%)': [
        84.62,
        36.57,
        27.98,
        36.04,
        44.64,
        32.83,
        38.01
    ],
    'Specificity (%)': [
        99.75,
        92.51,
        91.41,
        92.24,
        92.27,
        91.79,
        92.57
    ]
}

results_df = pd.DataFrame(results)

print("\nPYTHON RESULTS:")
print("="*70)
print(results_df.to_string(index=False))

print("\n\nR RESULTS (from your paper):")
print("="*70)
r_results = """
Algorithm                 Accuracy (%)  Sensitivity (%)  Specificity (%)
Random Forest                   84.22            87.91            54.83
Standard IF                     85.45            92.36            30.47
Single-Variable IF              85.70            92.52            31.45
SCiForest                       84.68            90.17            30.98
"""
print(r_results)

print("\n" + "="*70)
print("KEY FINDINGS:")
print("="*70)
print("\n1. RANDOM FOREST:")
print(f"   Python: Acc=98.14%, Sens=84.62%, Spec=99.75%")
print(f"   R:      Acc=84.22%, Sens=87.91%, Spec=54.83%")
print(f"   → Python MUCH better! (+14% accuracy, +45% specificity)")

print("\n2. ISOLATION FOREST VARIANTS:")
print(f"   Python: High specificity (91-92%), LOW sensitivity (28-45%)")
print(f"   R:      Low specificity (30-31%), HIGH sensitivity (90-92%)")
print(f"   → Completely OPPOSITE performance profiles!")

print("\n3. WINNER:")
print(f"   Python: Random Forest dominates ALL metrics")
print(f"   R:      Isolation Forest variants had best sensitivity")

print("\n" + "="*70)

FINAL RESULTS - ALL MODELS

PYTHON RESULTS:
         Algorithm  Accuracy (%)  Sensitivity (%)  Specificity (%)
     Random Forest         98.14            84.62            99.75
       Standard IF         86.54            36.57            92.51
Single-Variable IF         84.63            27.98            91.41
         SCiForest         86.25            36.04            92.24
  Density-Aware IF         87.20            44.64            92.27
          Boxed IF         85.50            32.83            91.79
   Fair-Cut Forest         86.75            38.01            92.57


R RESULTS (from your paper):

Algorithm                 Accuracy (%)  Sensitivity (%)  Specificity (%)
Random Forest                   84.22            87.91            54.83
Standard IF                     85.45            92.36            30.47
Single-Variable IF              85.70            92.52            31.45
SCiForest                       84.68            90.17            30.98


KEY FINDINGS:

1. RANDOM 